In [2]:
'''
    Don't Run this on conda environment.
'''
import os
import pandas as pd
import datasets
from datasets import load_dataset
from googletrans import Translator
import pickle as pkl
from tqdm.auto import tqdm

In [ ]:
translator = Translator()

In [4]:
nepCov19 = load_dataset("raygx/NepCov19Tweets")['train']

Found cached dataset parquet (C:/Users/Dell/.cache/huggingface/datasets/raygx___parquet/raygx--NepCov19Tweets-11ff376905985191/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
bdf = nepCov19.filter(lambda x: x['Sentiment']==0).to_pandas(batched=True,batch_size=50)

In [2]:
print("Step 1: Translating Nepali To English")
trans_en = []
for batch in tqdm(bdf):
    trans_en += translator.translate(batch['Sentences'].to_list(),src='ne',dest='en')

Found cached dataset parquet (C:/Users/Dell/.cache/huggingface/datasets/raygx___parquet/raygx--NepCov19Tweets-11ff376905985191/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\Dell\.cache\huggingface\datasets\raygx___parquet\raygx--NepCov19Tweets-11ff376905985191\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-f7fa500983a7f39c.arrow


Step 1: Translating Nepali To English


0it [00:00, ?it/s]

In [4]:
pd.DataFrame({'en':[x.text for x in trans_en]}).to_csv("nepCov19_neutral_googletrans_ne_2_en.csv")
trans_en = datasets.Dataset.from_dict({'en':[x.text for x in trans_en]}).to_pandas(batched=True,batch_size=50)

In [31]:
if os.path.exists("nepCov19_neutral_googletrans_ne_2_en.csv"):
    trans_en = pd.read_csv("nepCov19_neutral_googletrans_ne_2_en.csv")[['en']]
    trans_en = datasets.Dataset.from_pandas(trans_en).to_pandas(batched=True,batch_size=50)

In [32]:
print("Step 2: Translating English To Nepali")
trans_ne = []
for batch in tqdm(trans_en):
    trans_ne += translator.translate(batch['en'].to_list(),src='en',dest='ne')

Step 2: Translating English To Nepali


0it [00:00, ?it/s]

In [33]:
trans_ne = pd.DataFrame({'ne':[x.text for x in trans_ne]})
print(trans_ne)
trans_ne.to_csv("nepCov19_neutral_googletrans_en_2_ne.csv")

                                                     en
0           Covid Update मा कतिवटा नमूना परीक्षण गरियो?
1                                       स्वतन्त्र कोविड
2     प्रारम्भिक धार्मिक भेला होस् वा जनसमूहबाट सरका...
3     कुवेतबाट फर्किएपछि गरिएको कोभिड परीक्षणको रिपो...
4     के हो कोरोना भाइरस, कोभिड, यसका लक्षणहरु के के...
...                                                 ...
1666  विश्व स्वास्थ्य संगठनले कोरोना भाइरसलाई नयाँ न...
1667            WHO ले दिएको कोरोनाको नयाँ नाम कोविड हो
1668    कोरोना भाइरस संक्रमणको आधिकारिक नाम अब कोविड हो
1669  विश्व स्वास्थ्य संगठन (WHO) ले नोबल कोरोना भाइ...
1670  विश्व स्वास्थ्य संगठनले कोरोना भाइरस संक्रमणको...

[1671 rows x 1 columns]


In [12]:
if os.path.exists("nepCov19_neutral_googletrans_ne_2_en.csv"):
    trans_ne = pd.read_csv("nepCov19_neutral_googletrans_en_2_ne.csv")[['ne']]
    
agg_neutral = pd.DataFrame({'ne':trans_ne['ne'].to_list()+nepCov19.filter(lambda x: x['Sentiment']==0)['Sentences']})
print(agg_neutral.shape)

Loading cached processed dataset at C:\Users\Dell\.cache\huggingface\datasets\raygx___parquet\raygx--NepCov19Tweets-11ff376905985191\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-f7fa500983a7f39c.arrow


(9742, 1)


In [15]:
agg_neutral = agg_neutral.drop_duplicates()
agg_neutral.to_csv("googletrans_augmented_data.csv")